In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('/content/OnlineRetail.csv')
#data.head()
data.info()
#data.describe()


FileNotFoundError: ignored

In [ ]:
# Drop rows with missing values
data.dropna(inplace=True)

# Convert the 'Customer ID' column to integers
data['InvoiceNo'] = data['InvoiceNo'].astype(object)

# Keep only relevant columns for collaborative filtering
data = data[['InvoiceNo', 'StockCode', 'Description', 'Quantity']]

In [ ]:
# Calculate user-item interactions (quantity of each product purchased by each customer)
user_item_interactions = data.pivot_table(index='InvoiceNo', columns='Description', values='Quantity', fill_value=0)

In [ ]:
user_item_interactions

NameError: ignored

In [ ]:
from surprise import Reader, Dataset, KNNWithMeans
from surprise.model_selection import train_test_split

# Load the data into the Surprise format
reader = Reader(rating_scale=(0, 10))  # Assuming the maximum quantity is 10
surprise_data = Dataset.load_from_df(data[['Customer ID', 'Description', 'Quantity']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

# Build the recommendation model using KNNWithMeans
model = KNNWithMeans(k=50, sim_options={'name': 'pearson', 'user_based': False})
model.fit(trainset)


ModuleNotFoundError: ignored

In [ ]:
# Function to get product recommendations for a given customer
def get_recommendations(customer_id, num_recommendations=5):
    items_bought_by_user = user_item_interactions.loc[customer_id]
    items_to_predict = user_item_interactions.columns[items_bought_by_user == 0]
    testset = [(customer_id, item, 0) for item in items_to_predict]
    predictions = model.test(testset)
    top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    recommended_items = [item[0] for item in top_recommendations]
    return recommended_items

# Example: Get top 5 product recommendations for a specific customer (e.g., Customer ID 12345)
customer_id = 12345
recommendations = get_recommendations(customer_id, num_recommendations=5)

# Display the recommended product descriptions
recommended_products = data[data['Description'].isin(recommendations)]['Description'].unique()
print("Recommended Products:")
print(recommended_products)
